In [3]:
import pandas as pd
import numpy as np
import datetime
import pandas as pd

In this final, you will be asked to analyze a data set containing historical reservations for a fine-dining restaurant that is stored in the file "data/Reservation_Data.csv".  This restaurant only accepts reservations booked online and hence does not accept walk in customers.   Each row in this data corresponds to a booked reservation.  The columns have the following meaning:
- reservation\_date: The date that the reservation was booked for. In other words, this is the date when the customer will dine.
- reservation\_time: The time that the reservation was booked for.
- reservation\_party\_size: The size of the party for the corresponding reservation, i.e. the number of diners.
- reservation_date_booked: the date on which the reservation was made.
- datetime\_booked: The date and time corresponding to when the reservation was made (in UTC). This column has missing values, which have been entered as "#N/A".  **The restaurant is located in a Pacific time zone.**

Your goal in this final will be to undertand how customers schedule reservations at this restaurant.


# Part 1 

Your first task is to read in the data and do the following:

- delete rows with missing values
- convert datetime_booked to a datetime column with Pacific time zone
- combine reservation_date and reservation time to create a new column called reservation_datetime that is a datetime column that has a Pacific time zone. So the final dataframe you return should have 6 columns in total.
- Only keep reservations made at the following 8 time slots: 17:30, 17:45, 18:00, 18:15, 20:45, 21:00, 21:15, 21:30

Return this modified version of the original data frame.

In [4]:
def Read_Data():
    
    df_res = pd.read_csv("data/Reservation_Data.csv", na_values=["#N/A"]).dropna()
    
    df_res['datetime_booked'] = pd.to_datetime(df_res['datetime_booked'])
    df_res['datetime_booked'] = df_res['datetime_booked'].dt.tz_localize('UTC').dt.tz_convert('US/Pacific')

    df_res['reservation_datetime'] = pd.to_datetime(df_res['reservation_date'] + ' ' + df_res['reservation_time'])
    df_res['reservation_datetime'] = df_res['reservation_datetime'].dt.tz_localize('US/Pacific')

    array = ['17:30:00', '17:45:00', '18:00:00', '18:15:00', '20:45:00', '21:00:00', '21:15:00', '21:30:00']
    df_res = df_res.loc[df_res['reservation_time'].isin(array)]
    return df_res

In [5]:
df_res = Read_Data()
assert df_res.shape==(3359,6)

In [6]:
assert np.isclose(df_res.datetime_booked.dt.hour.mean(), 13.55969)

# Part 2 

In this next part, we will write two functions to understand basic patterns in the data.

The first function takes as input one of the eight reservation time slots that we are considering as a datetime time and it outputs the day of the week (as a string) with the smallest average party size of all reservation made for the given inputted time slot.

In [7]:
def Get_Avg_Party_Size(res_time):
    
    smallest_dow = None
    df_res = Read_Data()
    df_res["reservation_date"] = pd.to_datetime(df_res['reservation_date'])
    df_res["day_of_week"] = df_res["reservation_date"].dt.day_name()
    df_res["reservation_time"] =  pd.to_timedelta(df_res['reservation_time'], unit='s')
    res_time = res_time.strftime("%H:%M:%S")
    low = df_res.loc[df_res['reservation_time'] == res_time].groupby("day_of_week").mean().idxmin()
    smallest_dow = low[0]
    
    return smallest_dow

In [8]:
res_time = datetime.time(hour = 17, minute=30)
assert Get_Avg_Party_Size(res_time)=="Sunday"

In [9]:
res_time = datetime.time(hour = 21, minute=0)
assert Get_Avg_Party_Size(res_time)=="Tuesday"

The second function I would like you to write takes as input a parameter called num_days (you may assume this is an integer), and it returns the fraction of reservations made by parties size 1/2 that are made at most num_days in advance, and then also the same fraction but just for parties of 3/4.

**NOTE: The number of days in advance that a reservation is made is (reservation_datetime - datetime_booked).**

In [10]:
def Booked_in_Advance(num_days):
    
    df_res = Read_Data()
    num_days = datetime.timedelta(days=num_days)
    party_one_two = None
    party_three_four = None
    
    df_res["party"] = df_res["reservation_party_size"].apply(lambda x: '0' if x in [1,2] else '1') 
    
    new_res = df_res[df_res["reservation_datetime"] - df_res["datetime_booked"] <= num_days] 

    df_p12 = new_res.apply(lambda x : True if x['reservation_party_size'] <= 2 else False, axis = 1) 
    p12 = df_res.apply(lambda x : True if x['reservation_party_size'] <= 2 else False, axis = 1)

    denom12 = len(p12[p12 == True].index) 
    one_or_two = len(df_p12[df_p12 == True].index) 

    party_one_two = one_or_two/denom12 

    p34 = df_res.apply(lambda x : True if x['reservation_party_size'] >= 3 else False, axis = 1)
    df_p34 = new_res.apply(lambda x : True if x['reservation_party_size'] >= 3 else False, axis = 1)

    three_or_four = len(df_p34[df_p34 == True].index)
    denom34 = len(p34[p34 == True].index)

    party_three_four = three_or_four/denom34

    return party_one_two, party_three_four


In [11]:
result = Booked_in_Advance(7)
assert np.isclose(result[0],0.27188)

In [12]:
result = Booked_in_Advance(100)
assert np.isclose(result[1],0.860534)

# Part 3

Let's assume that there are two service periods:

- Service Period 1 consists of time slots 17:30, 17:45, 18:00, 18:15
- Service Period 2 consiss of time slots 20:45, 21:00, 21:15, 21:30

The next function takes as input res_time, which is one of the eight times above as a datetime time.  Given the inputted time, you will exclusively focus this analysis on either Service Period 1 or 2. For example, if the inputted res_time is 17:45, you should only consider reservations made during Service Period 1. For each day, find the first reservation that books at res_time (datetime_booked tells you the order in which reservations were scheduled).  Within the given Service Period of interest, compute how many reservations have already been booked for the given day before this reservation for res_time was made.  If $k-1$ reservations have already been made, this means that res_time was first booked as the $k$-th reservation.  We will say the "rank" of reservation time res_time on this day was $k$. So, if res_time was the first time booked on a particular day frot he relevant Service Period, its rank on this day is 1.  Return the average rank of res_time across all days in which a reservation for res_time was booked.


In [13]:
def Get_Avg_Rank(res_time):

    df_res = Read_Data()
    avg_Rank=None
    period1 = ['17:30:00', '17:45:00', '18:00:00', '18:15:00']
    period2 = ['20:45:00', '21:00:00', '21:15:00', '21:30:00']

    res_time = res_time.strftime("%H:%M:%S")
    for i in period1:
        if res_time in period1:
            df_res = df_res.loc[df_res['reservation_time'].isin(period1)]

    for j in period2:
        if res_time in period2:
            df_res = df_res.loc[df_res['reservation_time'].isin(period2)]

    groups = df_res.sort_values(by=["datetime_booked"]).groupby('reservation_date')
    count = 0
    total_rank = 0
    for name, group in groups:
        group = group.reset_index()
        k = group.loc[group['reservation_time'] == res_time].sort_values(by=["datetime_booked"]).head(1)
        if not k.empty: 
            rank = group.loc[group['reservation_time'] == res_time]\
                    .sort_values(by=["datetime_booked"]).head(1).index.values.astype(int)[0]
            count = count + 1
            total_rank = total_rank + rank +1

    avg_rank = total_rank/count
    return avg_rank

In [14]:
res_time = datetime.time(hour = 21, minute=30)
assert np.isclose(Get_Avg_Rank(res_time),3.9035 )

In [15]:
res_time = datetime.time(hour = 20, minute=45)
assert np.isclose(Get_Avg_Rank(res_time),1.45714)